In [0]:
# Lib Imports
import os

def main():
  
  # 1. Set Parameters
  src_path = "/Volumes/workspace/default/trip_record_data/trip_record_data/"
  schema_path = "workspace.default"
  table_name = ["yellow_tripdata", "green_tripdata", "fhv_tripdata", "fhvhv_tripdata"]
  limit_files = None # Set a limit to the number of files to process / 'None' for all

  # 2. Read/Sort Files
  all_src_files = os.listdir(src_path)
  all_src_files = sorted(all_src_files, reverse=True)
  sort_src_files = []
  for table in table_name:
    sorted_src = [table]
    sorted_src.append([f for f in all_src_files if f.startswith(table)])
    if limit_files:
      sorted_src[1] = sorted_src[1][:limit_files]
    sort_src_files.append(sorted_src)
  
  # 3. Load Data
  for table in sort_src_files:
    table_name = table[0]
    src_files = table[1]
    table_path = schema_path + ".stg_" + table_name
    spark.sql(f"TRUNCATE TABLE {table_path}")

    for file in src_files:
      df = spark.read.parquet(src_path+"/"+file)
      (
        df.write
          .format("delta")
          .mode("overwrite")
          .option("overwriteSchema", "true")
          .saveAsTable(table_path)
      )

main()